# Retrieval Test

In [1]:
import sys
import os
sys.path.append('/home/glen/2025-LLM-Project')

from src.data_loader import data_load, data_process, data_chunking
from src.vector_db import generate_vector_db, load_vector_db

In [2]:
from main import generate_index_name

df = data_load("data/data_list.csv")
df = data_process(df, apply_ocr=False, file_type="all")
all_chunks = data_chunking(df=df, splitter_type="recursive", size=300, overlap=50)
index_name = "all_100_recursive_KoE5_faiss_index"
embeddings = generate_vector_db(all_chunks, "nlpai-lab/KoE5")
vector_store = load_vector_db("../data", "nlpai-lab/KoE5")

서울특별시_2024년 지도정보 플랫폼 및 전문활용 연계 시스템 고도화 용.pdf:  56%|█████▌    | 42/75 [00:01<00:02, 15.43it/s]

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict



기초과학연구원_2025년도 중이온가속기용 극저온시스템 운전 용역.pdf: 100%|██████████| 49/49 [00:00<00:00, 304.99it/s]


TypeError: generate_vector_db() missing 1 required positional argument: 'index_name'

In [ ]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

In [ ]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

bm25_retriever = BM25Retriever.from_documents(all_chunks)
bm25_retriever.k = 3

In [ ]:
hybrid_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, retriever],
    weights=[0.5, 0.5],  # Equal weight for hybrid fusion
)

In [ ]:
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain.retrievers import ContextualCompressionRetriever

reranker = ContextualCompressionRetriever(
    base_retriever=hybrid_retriever,
    base_compressor=EmbeddingsFilter(
        embeddings=embeddings,
        similarity_threshold=0.45
    )
)

query = "배드민턴장 및 탁구장 예약방법"
docs = reranker.invoke(query)

# Print top 3 compressed docs
for i, doc in enumerate(docs[:3]):
    print(f"--- 📄 문서 {i+1} ---")
    print(doc.page_content[:500])
    print(doc.metadata)